In [19]:
import pandas as pd
import numpy as np
from mystuff import bdp,bdh
import datetime
import funciones_optim
from importlib import reload
reload(funciones_optim)
from funciones_optim import backt

In [32]:
# carga precios de bloomberg 
p = backt.loadprices(20171231)

In [33]:
#calculo de retornos para cada asset class
rt = p.apply(backt.rt,args=[p],axis=0).drop(['USCASH'],axis=1)
rt = rt.loc[:,['SPX','EEM','XAU','USTTEN','COCASH','USDCOP']]
r = rt.drop(['COCASH'],axis=1)

In [34]:
#rango de fechas donde se va a hacer el backtesting
sdate = datetime.datetime(2019,1,31)
fdate = datetime.datetime(2023,12,31)

#Ejemplo de un portafolio equally weighted que rebalancea mensualmente y vuelve siempre al vector inicial
w = np.ones(5)*0.20
rdates =pd.date_range(sdate,fdate)
rbw = pd.DataFrame(np.tile(w,(len(rdates),1)),index=rdates).resample('M').last()
wp,rp = backt.portr(r,sdate,fdate,w,rebal=False)

In [35]:
# ejemplo de 5 portafolios con rebalanceos mensuales a recomendaciones ( en este caso los w son generados aleateoriamente como proxy de un modelo)
# wi es el vector inicial del modelo
# rbw contiene los vectores recomendacion mensuales
ports=[]
for _ in range(5):
    wx = pd.DataFrame(np.random.rand(5,len(rbw)))
    wx = wx/wx.iloc[:-1,:].sum()
    wx = wx.transpose()
    wx.index = rbw.index
    wi = np.random.rand(5)
    wi = wi/sum(wi[:-1])
    w_,r_ = backt.portr(r,sdate,fdate,wi,rebal=True,rbw=wx)
    ports.append(r_)
ports = pd.concat(ports,axis=1)
ports.columns = [f'port {c}' for c in ports.columns]


In [36]:
# devuelve un dataframe como metricas de desempeño y riesgo de los portafolios.
df = backt.pmetrics(ports,rt.COCASH,bench=0)

In [43]:
#un ejemplo de dataframe comparativo en metricas de portafolio. 
df.round(3)
#ports.add(1).apply(np.log).cumsum().apply(np.exp).mul(100).plot()

,port 0,port 1,port 2,port 3,port 4
ret,33.518,44.048,37.925,51.838,37.444
retm,0.532,0.651,0.580,0.742,0.577
volm,11.029,9.970,10.250,10.374,10.607
sharpe,0.002,0.014,0.007,0.022,0.006
mdd,-18.432,-19.123,-19.786,-14.212,-21.250
etg/etl,1.408,1.217,1.401,1.355,1.164
downdev,2.810,2.565,2.619,2.620,2.457
sortino,0.189,0.254,0.221,0.283,0.235
